In [ ]:
import pandas as pd
import numpy as np
# import pywedge as pw
from pathlib import Path
# from data_dashboard import Dashboard

import matplotlib.pyplot as plt
# import seaborn as sns

# import lightgbm
# import sklearn.linear_model as lm
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from ydata_profiling import ProfileReport
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
import plotly.express as px

from pylab import savefig
# import shap

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
path_to_data = Path('/home/kosintsev_ag@sib.evraz.com/PycharmProjects/2023_evraz_hackaton/exploration/data/kafka_data.csv')
data = pd.read_csv(path_to_data)
mapping = pd.read_excel('/home/kosintsev_ag@sib.evraz.com/PycharmProjects/2023_evraz_hackaton/exploration/data/Маппинг сигналов.xlsx')
mapping.index = mapping['Код сигнала в Kafka']
mapping.drop('Код сигнала в Kafka', axis=1, inplace=True)

In [ ]:
mapping_dict = mapping.to_dict()['Название']
print(str(mapping_dict)[:200])

In [ ]:
print('Найден мапинг для датчиков:', len([True for i in set(data.columns) if i in set(mapping_dict.keys())]))
missed_map_cols = [i for i in set(data.columns) if i not in set(mapping_dict.keys())]
print('Не найден мапинг для датчиков:', missed_map_cols)

In [ ]:
print('shape:\n', data.shape)
data = data.rename(columns=mapping_dict)
data.index = data['moment']
# data.drop('moment', axis=1, inplace=True)
data.drop(missed_map_cols, axis=1, inplace=True)

print('shape:\n', data.shape)
print('\n', data.info())
data.head(5)

In [ ]:
data.describe()

In [ ]:
def plot_scatters(df, plot_cols):
    for col in plot_cols:
        fig, ax = plt.subplots(figsize=(10,10))
        ax.set_title(col)
        ax.scatter(df.index, df[col], c=[[0.5, 0, 0.5, 0.15]])

def plot_hists(df, plot_cols):
    for col in plot_cols:
        f, ax = plt.subplots(figsize=(10,6))
        ax.set_title(col)
        df[col].hist(bins=100)

def isna_percent(df):
    return (df.isna().sum()/df.shape[0]*100).sort_values(ascending=False)

def isna_count(df):
    return df.isna().sum().sort_values(ascending=False)

In [ ]:
isna_count(data)

In [ ]:
data = data.dropna().sort_index()
data.head()

In [ ]:
all_ustavki = [i for i in data.columns if 'уставк' in i.lower()]
us_ravni = (data[all_ustavki] == data[all_ustavki].iloc[0]).min()
us_ravni[us_ravni==False]

In [ ]:
a = data['Уставка аварийная max ГВ 7т Эксгаустер №4'] - data['Уставка аварийная max ГВ 7т Эксгаустер №4'].shift(-1)
a[a!=0]

##### Всего 1 уставка отличается в истории, менялась туда обратно 1 раз. Все уставки сбросим с data

In [ ]:
ustavki = data[all_ustavki].copy()
data.drop(all_ustavki, axis=1, inplace=True)
data.index = data.index.astype('datetime64[ns]')
data.columns = data.columns.str.lower()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data_exh = dict()

In [ ]:
for exh_id in range(1, 7):
    print(exh_id)
    exhauster_i_cols = [
        i for i in data.columns if f'ксгаустер №{exh_id}' in i
                                   or f'ксгаустера №{exh_id}' in i
                                   or f'ксгаустером №{exh_id}' in i
    ]
    print(f'столбцов для эксгаустера #{exh_id}', len(exhauster_i_cols))

    data_exh[exh_id] = dict()
    data_exh[exh_id]['data'] = data[exhauster_i_cols].copy()
    print('размер df:', data_exh[exh_id]['data'].shape)

    targets_i = [i for i in data_exh[exh_id]['data'].columns if 'вибрация' in i and 'уставка' not in i]
    data_exh[exh_id]['targets'] = targets_i


In [ ]:
# mc = pw.Pywedge_Charts(data_3exh, c=None, y=targets_3[-2])
# charts = mc.make_charts()

In [ ]:
# output_directory = 'your_path/dashboard_output'
# X, y, descriptions = data_3exh.drop(targets_3[-2], axis=1), data[targets_3[-2]], None
# # descriptions is an optional argument, more in documentation
# dsh = Dashboard(X, y, output_directory, descriptions)
# dsh.create_dashboard()

#### Pandas profiling

In [ ]:
# profile = ProfileReport(data_3exh, title=f"{exhauster_number} exhauster")
# profile.to_file("3_exhauster.html")

#### Проанализируем поведение вибраций по подшипникам связных эксгаустеров

In [ ]:
px.line(data_exh[0]['data'])

# Обучение моделей

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop('M25', axis=1), data['M25'], test_size=0.2, shuffle=False)

model = Fedot(problem='regression', timeout=15, preset='best_quality', n_jobs=-1)
model.fit(features=x_train, target=y_train)
prediction = model.predict(features=x_test)
metrics = model.get_metrics(target=y_test)

In [ ]:
metrics

In [ ]:
r2_score(y_test, prediction)

In [ ]:
cols_for_model = []

gbm = lightgbm.LGBMRegressor()
model_list = [gbm, ]


def train_model(model_list, x, y):
    for model in model_list:

        df_r2 = pd.Series()

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)

        fig, ax = plt.subplots(figsize=(12,8))
        ax.set_title(f"Модель {model}")
        feature_importances = pd.Series(model.feature_importances_, index=x.columns)
        top_vals = feature_importances.nlargest(10).sort_values(ascending=True)
        y_label = [str(i) for i in top_vals.index.values]
        # print(y_label)
        ax.barh(y_label, top_vals.values)
        plt.show()

        mse, mae = np.round(mean_absolute_error(y_test, y_pred),10), np.round(mean_squared_error(y_test, y_pred), 10)
        mse = np.round(mse, 8)
        mae = np.round(mae, 8)

        print("MSE =", mse)
        print("MAE =", mae)
        plt.style.use("ggplot")
        fig, ax = plt.subplots(figsize=(15,15))

        test_r2 = np.round(r2_score(y_test, y_pred)*100, 2)
        labell = f"R2 = {test_r2}%, MAE={mae}"
        ax.set_title(labell)
        ax.set_xlabel("FACT")
        ax.set_ylabel("PREDICTION")
        ax.scatter(y_test, y_pred, c=[[0.5, 0, 0.5, 0.15]],)

        line_45_x = [i for i in [y_test.min(), y_test.max()]]
        line_45_y = [i for i in [y_test.min(), y_test.max()]]
        ax.plot(line_45_x, line_45_y, color="gray")
        df_r2[str(model)] = test_r2
        plt.show()
        print("R2 =", test_r2)